In [8]:
from transformers import AutoModelForCausalLM, AutoTokenizer, TextDataset, Trainer, TrainingArguments

In [9]:
import torch
torch.cuda.is_available()

True

In [41]:
model_name = "TinyLlama/TinyLlama-1.1B-Chat-v1.0"
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(model_name)

/home/sined/micromamba/envs/nlp_venv/lib/python3.12/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [50]:
train_dataset = TextDataset(
    tokenizer=tokenizer,
    file_path="sentences.txt",
    block_size=128
)

/home/sined/micromamba/envs/nlp_venv/lib/python3.12/site-packages/transformers/data/datasets/language_modeling.py:53: FutureWarning: This dataset will be removed from the library soon, preprocessing should be handled with the 🤗 Datasets library. You can have a look at this example script for pointers: https://github.com/huggingface/transformers/blob/main/examples/pytorch/language-modeling/run_mlm.py
  warnings.warn(


In [51]:
training_args = TrainingArguments(
    output_dir="./output",
    overwrite_output_dir=True,
    num_train_epochs=3,
    per_device_train_batch_size=4,
    save_steps=10_000,
    save_total_limit=2,
)

In [52]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_dataset
)


In [53]:
trainer.train()

ValueError: The batch received was empty, your model won't be able to train on it. Double-check that your training dataset contains keys expected by the model: input_ids,attention_mask,position_ids,past_key_values,inputs_embeds,labels,use_cache,output_attentions,output_hidden_states,return_dict,cache_position,label,label_ids,labels.